In [27]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from colabgymrender.recorder import Recorder

Load The Enviorment

In [13]:
enviorment_name = 'CartPole-v0'
env = gym.make(enviorment_name)
env = Recorder(env, './video')

c:\Users\auton\AppData\Local\Programs\Python\Python39\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


In [15]:
# Test the enviorment
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:1 Score:26.0
Episode:2 Score:21.0
Episode:3 Score:47.0
Episode:4 Score:30.0
Episode:5 Score:19.0


Understanding the enviorment

In [19]:
env.action_space

Discrete(2)

In [20]:
env.action_space.sample()

0

In [17]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [21]:
env.observation_space.sample()

array([1.6141360e+00, 2.5845854e+38, 2.0086285e-01, 1.3051264e+38],
      dtype=float32)

Observation (Box(4)):
    0 - Cart position: -4.8 to 4.8
    1 - Cart velocity: -inf to inf
    2 - Pole angle: -0.418 rad to 0.418 rad
    3 - Pole angular velocity: -inf to inf

Action(Discret(2)):
    0 - Left
    1 - Right

Traing RL Model

In [22]:
log_path = os.path.join('Training', 'Logs')
log_path

'Training\\Logs'

In [29]:
env = gym.make(enviorment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [31]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 2961 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1815        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008320468 |
|    clip_fraction        | 0.0472      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.549      |
|    explained_variance   | 0.235       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.92        |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00422    |
|    value_loss           | 9.24        |
-----------------------------------------
---

Save and Reload the Model

In [32]:
# save the model
PPO_Path = os.path.join('Training', 'saved_models', 'PPO_model_CartPole')
model.save(PPO_Path)

In [33]:
# Load the model
del model
model = PPO.load(PPO_Path, env=env)

Evaluation

In [36]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

c:\Users\auton\AppData\Local\Programs\Python\Python39\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

Testing the Model   

In [37]:
# Test the enviorment
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


Viewing Logs on TensorBoard 

In [38]:
training_log_path = os.path.join(log_path, 'PPO_1')
training_log_path

'Training\\Logs\\PPO_1'

In [ ]:
!tensorboard --logdir={training_log_path}

Adding a Callback on the training stage

In [39]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [40]:
save_path = os.path.join('Training', 'saved_models')
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                            callback_on_new_best=stop_callback, 
                            eval_freq=10000, 
                            best_model_save_path=save_path, 
                            verbose=1)

In [41]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [42]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 2938 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1790         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0077503882 |
|    clip_fraction        | 0.0803       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | -0.000329    |
|    learning_rate        | 0.0003       |
|    loss                 | 7.57         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0119      |
|    value_loss           | 56.4         |
----------------------------

c:\Users\auton\AppData\Local\Programs\Python\Python39\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=198.20 +/- 3.60
Episode length: 198.20 +/- 3.60
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 198          |
|    mean_reward          | 198          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0066585923 |
|    clip_fraction        | 0.0571       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.603       |
|    explained_variance   | 0.238        |
|    learning_rate        | 0.0003       |
|    loss                 | 29.6         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0162      |
|    value_loss           | 62.1         |
------------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 1478  |
|    iterations      |

Using an atlernate algotirthm

In [43]:
from stable_baselines3 import DQN

In [44]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [45]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.959    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 8787     |
|    time_elapsed     | 0        |
|    total_timesteps  | 86       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.929    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 10143    |
|    time_elapsed     | 0        |
|    total_timesteps  | 150      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.903    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 9892     |
|    time_elapsed     | 0        |
|    total_timesteps  | 204      |
----------------------------------
------------------------

c:\Users\auton\AppData\Local\Programs\Python\Python39\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 564      |
|    fps              | 12728    |
|    time_elapsed     | 0        |
|    total_timesteps  | 12245    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 568      |
|    fps              | 12754    |
|    time_elapsed     | 0        |
|    total_timesteps  | 12384    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 572      |
|    fps              | 12762    |
|    time_elapsed     | 0        |
|    total_timesteps  | 12469    |
----------------------------------
----------------------------------
| rollout/          

In [47]:
DQN_path = os.path.join('Training', 'saved_models', 'DQN_model_CartPole')
model.save(DQN_path)